In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks
from sklearn.metrics import confusion_matrix, classification_report

import warnings
warnings.filterwarnings("ignore")

print(f"TensorFlow version: {tf.__version__}")

2026-02-24 13:06:41.720212: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


TensorFlow version: 2.18.0


In [2]:
# Paths
processed_dir = Path("../data/processed")
splits_dir = Path("../data/splits")
models_dir = Path("../models")
models_dir.mkdir(parents=True, exist_ok=True)

# Load preprocessing config
with open(processed_dir / "preprocessing_config.json") as f:
    config = json.load(f)

# Load class mappings
with open(processed_dir / "class_to_index.json") as f:
    class_to_index = json.load(f)

with open(processed_dir / "index_to_class.json") as f:
    index_to_class = {int(k): v for k, v in json.load(f).items()}

# Load class weights
with open(processed_dir / "class_weights.json") as f:
    class_weights = {int(k): v for k, v in json.load(f).items()}

# Core config
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
N_CLASSES = len(class_to_index)
MEAN = config['normalization']['mean']
STD = config['normalization']['std']
RANDOM_SEED = config['random_seed']

print(f"Classes: {N_CLASSES}")
print(f"Image size: {IMAGE_SIZE}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Normalization mean: {MEAN}")
print(f"Normalization std: {STD}")

Classes: 15
Image size: (224, 224)
Batch size: 32
Normalization mean: [0.46, 0.48, 0.42]
Normalization std: [0.21, 0.18, 0.22]
